# RAG using Upstage Layout Analysis and Groundedness Check
This example illustrates RAG using [Upstage](https://python.langchain.com/docs/integrations/providers/upstage/) Layout Analysis and Groundedness Check.

In [ ]:
from typing import List

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.documents.base import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.base import RunnableSerializable
from langchain_upstage import ChatUpstage, GroundednessCheck, UpstageEmbeddings, UpstageLayoutAnalysisLoader

model = ChatUpstage()

file_path = "/PATH/TO/YOUR/FILE.pdf"

files = [file_paths]

vectorstore = ...

for f in files:
    loader = UpstageLayoutAnalysisLoader(file_path=f, split="element")
    docs = loader.load()
    vectorstore.add_documents(docs)
    

vectorstore = DocArrayInMemorySearch.from_documents(docs, embedding=UpstageEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

retrieved_docs = retriever.get_relevant_documents("How many parameters in SOLAR model?")

groundedness_check = GroundednessCheck()
groundedness = ""
while groundedness != "grounded":
    chain: RunnableSerializable = RunnablePassthrough() | prompt | model | output_parser

    result = chain.invoke(
        {
            "context": retrieved_docs,
            "question": "How many parameters in SOLAR model?",
        }
    )

    # convert all Documents to string
    def formatDocumentsAsString(docs: List[Document]) -> str:
        return "\n".join([doc.page_content for doc in docs])

    groundedness = groundedness_check.run(
        {
            "context": formatDocumentsAsString(retrieved_docs),
            "assistant_message": result,
        }
    ).content